In [ ]:
from google.colab import drive

drive.mount('/MyDrive')

!ls

Mounted at /MyDrive
sample_data


In [ ]:
from keras.engine import input_layer
import numpy as np
import cv2
import tensorflow as tf
from keras.losses import sparse_categorical_crossentropy
from keras.optimizers import Adam
from matplotlib import pyplot as plt
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

data_path = '/MyDrive/MyDrive/alzheimer/dataset/Task_01_with/'
fig_path = '/MyDrive/MyDrive/alzheimer/fig/'
model_path = '/MyDrive/MyDrive/alzheimer/model/'

In [ ]:
batch_size = 32
img_width, img_height, img_num_channels = 299, 299, 3
no_classes = 2
no_epochs = 100
optimizer = Adam()
verbosity = 1

data = tf.keras.utils.image_dataset_from_directory(directory = data_path, image_size=(299, 299))
data_iterator = data.as_numpy_iterator()
batch = data_iterator.next()

data = data.map(lambda x,y: (x/255,y))
scaled_iterator = data.as_numpy_iterator()

batch = scaled_iterator.next()

print(len(data))

train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

Found 664 files belonging to 2 classes.
21


In [ ]:
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications

base_model = applications.InceptionV3(weights='imagenet', 
                                      classes = no_classes,
                                include_top=False, 
                               
                                input_shape=(299, 299,3))
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))
add_model.add(BatchNormalization())
add_model.add(Dense(512, activation='relu'))
add_model.add(Dense(1, 
                    activation='sigmoid'))

dnn_model = add_model
dnn_model.compile(loss=tf.losses.BinaryCrossentropy(), 
              optimizer=optimizers.SGD(lr=1e-4, 
                                      momentum=0.9),
              #optimizer = Adam(),
              metrics=['accuracy'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

dnn_model.summary()



Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 8, 8, 2048)        21802784  
                                                                 
 global_average_pooling2d_20  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dropout_9 (Dropout)         (None, 2048)              0         
                                                                 
 batch_normalization_2150 (B  (None, 2048)             8192      
 atchNormalization)                                              
                                                                 
 dense_27 (Dense)            (None, 512)               1049088   
                                                                 
 dense_28 (Dense)            (None, 1)               

In [ ]:
python_history = dnn_model.fit(
        train,
        epochs=50,
        validation_data=val,
        callbacks=[callback],
        batch_size=batch_size,
        verbose=verbosity
    )

Epoch 1/50
14/14 [==============================] - 8s 433ms/step - loss: 0.7098 - accuracy: 0.6138 - val_loss: 0.5877 - val_accuracy: 0.7500
Epoch 2/50
14/14 [==============================] - 6s 374ms/step - loss: 0.7014 - accuracy: 0.6049 - val_loss: 0.6028 - val_accuracy: 0.7109
Epoch 3/50
14/14 [==============================] - 5s 295ms/step - loss: 0.7205 - accuracy: 0.5759 - val_loss: 0.5681 - val_accuracy: 0.7344
Epoch 4/50
14/14 [==============================] - 4s 285ms/step - loss: 0.6640 - accuracy: 0.6496 - val_loss: 0.6349 - val_accuracy: 0.6797
Epoch 5/50
14/14 [==============================] - 5s 315ms/step - loss: 0.7069 - accuracy: 0.5960 - val_loss: 0.5970 - val_accuracy: 0.7031
Epoch 6/50
14/14 [==============================] - 4s 282ms/step - loss: 0.6537 - accuracy: 0.6540 - val_loss: 0.5761 - val_accuracy: 0.7422
Epoch 7/50
14/14 [==============================] - 8s 522ms/step - loss: 0.6704 - accuracy: 0.6362 - val_loss: 0.5800 - val_accuracy: 0.7031
Epoch 

In [ ]:
dnn_model.save(model_path + 'inception_new.h5', overwrite=True)

In [ ]:
test_dict = dnn_model.evaluate(
    test,
    steps=test_size,
    return_dict=True
)

2/2 [==============================] - 2s 158ms/step - loss: 0.6251 - accuracy: 0.6562


In [ ]:
dnn_model.metrics_names
 #  "Accuracy"
plt.plot(python_history.history['accuracy'])
plt.plot(python_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.savefig(fig_path + 'accuracy_yt.jpg')
    # "Loss"
plt.plot(python_history.history['loss'])
plt.plot(python_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.savefig(fig_path + 'loss_yt.jpg')

In [ ]:
import tensorflow as tf
import os
#model = tf.keras.models.load_model("./new_model_spero_fighissimo")
converter = tf.lite.TFLiteConverter.from_keras_model(dnn_model)

tflite_quant_model = converter.convert()#save converted quantization model to tflite format
open(model_path + "model_inception.tflite", "wb").write(tflite_quant_model)

91337436